## Importações

In [163]:
import pandas as pd
from math import floor
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import numpy as np
from scipy.stats import mstats

## Extraindo a Base

In [164]:
df_original = pd.read_csv("..\dados\ehresp_2014.csv")

df = df_original.copy()

display(df)

,tucaseid,tulineno,eeincome1,erbmi,erhhch,erincome,erspemch,ertpreat,ertseat,ethgt,...,eumeat,eumilk,euprpmel,eusoda,eustores,eustreason,eutherm,euwgt,euwic,exincome1
0,20140101140007,1,-2,33.200001,1,-1,-1,30,2,0,...,1,2,1,-1,2,1,2,170,1,2
1,20140101140011,1,1,22.700001,3,1,-1,45,14,0,...,1,2,1,-1,1,2,2,128,2,0
2,20140101140028,1,2,49.400002,3,5,-1,60,0,0,...,-1,-1,2,2,-1,-1,-1,270,2,12
3,20140101140063,1,-2,-1.000000,3,-1,-1,0,0,0,...,2,2,1,1,2,6,-1,-2,2,2
4,20140101140168,1,2,31.000000,3,5,-1,65,0,0,...,1,2,1,2,1,1,2,210,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11207,20141212142472,1,1,27.400000,3,1,-1,45,0,0,...,1,2,1,1,1,2,2,180,-1,0
11208,20141212142476,1,2,24.000000,3,5,5,30,0,0,...,1,2,3,2,1,2,2,140,2,0
11209,20141212142480,1,1,22.200001,3,1,5,90,5,0,...,1,2,1,-1,3,3,2,142,-1,0
11210,20141212142484,1,2,23.400000,3,5,-1,240,0,0,...,-1,-1,2,1,-1,-1,-1,168,2,0


## Colunas Escolhidas

In [165]:
df = df[['euexfreq','eugenhth','eufastfdfrq','eudietsoda','eusoda','euwgt','euhgt']]
df

,euexfreq,eugenhth,eufastfdfrq,eudietsoda,eusoda,euwgt,euhgt
0,-1,1,-1,-1,-1,170,60
1,-1,2,1,-1,-1,128,63
2,-1,5,-1,-1,2,270,62
3,-1,2,-1,2,1,-2,64
4,5,4,-1,-1,2,210,69
...,...,...,...,...,...,...,...
11207,2,2,3,2,1,180,68
11208,-1,3,-2,-1,2,140,64
11209,2,1,4,-1,-1,142,67
11210,1,1,3,1,1,168,71


## Ajeitando a Base

Para começar, eu verifico os valores da base com os valores do pdf (dicionário da base)

In [166]:
df['eusoda'].value_counts()

 2    4470
-1    3695
 1    3043
-2       4
Name: eusoda, dtype: int64

In [167]:
df['eudietsoda'].value_counts()

-1    8169
 2    1780
 1    1181
 3      76
-2       4
-3       2
Name: eudietsoda, dtype: int64

Não são descritos valores negativos

Se somarmos a quantidade dos valores negativos na coluna "eusoda" com os valores 2 (quem não bebe refrigerante) da o valor do -1 da coluna "eudietsoda". Isso quer dizer que o -1 significa quem não bebe refri

In [168]:
df['eudietsoda'].replace({-1: 0}, inplace=True)
df['eudietsoda'].value_counts()

 0    8169
 2    1780
 1    1181
 3      76
-2       4
-3       2
Name: eudietsoda, dtype: int64

Não consigo dizer quais são os outros valores negativos

In [169]:
df = df.loc[df['eudietsoda'] >= 0]
df['eudietsoda'].value_counts()

0    8169
2    1780
1    1181
3      76
Name: eudietsoda, dtype: int64

Agora que já tenho a informação de quem bebe refri, não preciso mais da coluna "eusoda"

In [170]:
df = df.drop("eusoda", axis=1)
df

,euexfreq,eugenhth,eufastfdfrq,eudietsoda,euwgt,euhgt
0,-1,1,-1,0,170,60
1,-1,2,1,0,128,63
2,-1,5,-1,0,270,62
3,-1,2,-1,2,-2,64
4,5,4,-1,0,210,69
...,...,...,...,...,...,...
11207,2,2,3,2,180,68
11208,-1,3,-2,0,140,64
11209,2,1,4,0,142,67
11210,1,1,3,1,168,71


Continuo estes processos com as demais colunas

In [171]:
df['euexfreq'].value_counts()

-1     4194
 3     1625
 2     1146
 7     1118
 4     1058
 5      912
 1      573
 6      353
 14      59
 10      40
 9       21
 8       19
-2       18
 21      16
 12      11
 11       7
 28       6
 15       5
 17       4
 20       4
-3        3
 35       2
 16       2
 25       2
 18       2
 13       2
 30       2
 38       1
 24       1
Name: euexfreq, dtype: int64

In [172]:
df['euexfreq'].replace({-1: 0}, inplace=True)
df = df.loc[df['euexfreq'] >= 0]
df['euexfreq'].value_counts()

0     4194
3     1625
2     1146
7     1118
4     1058
5      912
1      573
6      353
14      59
10      40
9       21
8       19
21      16
12      11
11       7
28       6
15       5
17       4
20       4
35       2
16       2
25       2
18       2
13       2
30       2
38       1
24       1
Name: euexfreq, dtype: int64

In [173]:
df['eugenhth'].value_counts()

 2    3751
 3    3480
 1    2015
 4    1365
 5     492
-2      36
-3      27
-1      19
Name: eugenhth, dtype: int64

In [174]:
df = df.loc[df['eugenhth'] >= 0]
df['eugenhth'].value_counts()

2    3751
3    3480
1    2015
4    1365
5     492
Name: eugenhth, dtype: int64

In [175]:
df['eufastfdfrq'].value_counts()

-1     4670
 1     2106
 2     1771
 3     1062
 4      535
 5      370
 7      248
 6      130
 10      66
 8       34
-2       28
 14      25
 12      18
 15      11
 9       10
 11       5
 20       4
 17       4
 21       4
 13       2
Name: eufastfdfrq, dtype: int64

In [176]:
df['eufastfdfrq'].replace({-1: 0}, inplace=True)
df = df.loc[df['eufastfdfrq'] >= 0]
df['eufastfdfrq'].value_counts()

0     4670
1     2106
2     1771
3     1062
4      535
5      370
7      248
6      130
10      66
8       34
14      25
12      18
15      11
9       10
11       5
20       4
17       4
21       4
13       2
Name: eufastfdfrq, dtype: int64

Para as ultimas colunas, de peso e altura será feito a validação das informações e a transformação das medidas

In [177]:
df = df.loc[df['euwgt'] >= 0]
df['euwgt'].value_counts()

180    520
160    479
150    463
170    448
200    406
      ... 
268      1
261      1
299      1
291      1
333      1
Name: euwgt, Length: 223, dtype: int64

In [178]:
# Conversão da coluna em libras para quilogramas
df['euwgt'] = (df['euwgt'] * 0.453592).round(2)
df['euwgt'].value_counts()

81.65     520
72.57     479
68.04     463
77.11     448
90.72     406
         ... 
121.56      1
118.39      1
135.62      1
132.00      1
151.05      1
Name: euwgt, Length: 223, dtype: int64

In [179]:
df = df.loc[df['euhgt'] >= 0]
df['euhgt'].value_counts()

66    966
64    949
67    922
65    832
68    767
62    755
63    753
70    747
69    672
72    649
71    641
61    412
60    376
73    306
74    264
75    132
59    124
76     94
77     68
56     54
58     46
57     28
Name: euhgt, dtype: int64

In [180]:
# Conversão da coluna com valores em polegadas para metros
df['euhgt'] = (df['euhgt'] * 0.0254).round(2)
df['euhgt'].value_counts()


1.68    966
1.63    949
1.70    922
1.65    832
1.73    767
1.57    755
1.60    753
1.78    747
1.75    672
1.83    649
1.80    641
1.55    412
1.52    376
1.85    306
1.88    264
1.90    132
1.50    124
1.93     94
1.96     68
1.42     54
1.47     46
1.45     28
Name: euhgt, dtype: int64

In [185]:
df

,euexfreq,eugenhth,eufastfdfrq,eudietsoda,euwgt,euhgt
0,0,1,0,0,77.11,1.52
1,0,2,1,0,58.06,1.60
2,0,5,0,0,122.47,1.57
4,5,4,0,0,95.25,1.75
5,2,3,3,1,99.79,1.80
...,...,...,...,...,...,...
11206,5,3,0,0,67.59,1.57
11207,2,2,3,2,81.65,1.73
11209,2,1,4,0,64.41,1.70
11210,1,1,3,1,76.20,1.80


Verifico se a informação de que a base de dados não possui dados nulos é correta.

In [184]:
df.isnull().values.any()

False

## Análise dos Dados

### Box Plots

In [188]:
for i in df.columns.tolist():
    fig = px.box(df, y=i, points= 'all')

    fig.show()

In [189]:
df.to_excel('..\dados\df.xlsx', index=False)